In [42]:
import pymysql

domain = ""
user = "sammy"
password = ""
db_name = "automation"

def read_employee_list():
    db = pymysql.connect(domain, user, password, db_name)
    # prepare a cursor object using cursor() method
    cursor = db.cursor()
    # Prepare SQL query to INSERT a record into the database.
    sql = "SELECT * FROM automation.employee"

    # Execute the SQL command
    cursor.execute(sql) 
    # disconnect from server
    db.close()
    # Fetch all the rows in a list of lists.
    return cursor.fetchall()


In [43]:
def create_employee(employee):
    db = pymysql.connect(domain, user, password, db_name)
    table_name="employee"

    cursor = db.cursor()

    # Prepare SQL query to INSERT a record into the database.
    sql = "INSERT INTO %s (`first_name`, `last_name`, `age`, `sex`, `date`) VALUES ('%s','%s', '%d', '%s', '%s')" \
          % (table_name, employee.get_first_name(), employee.get_last_name(),\
             employee.get_age(), employee.get_sex(),employee.get_onboard_date())
    # try:
    # Execute the SQL command
    cursor.execute(sql)

    cursor.close()
    db.commit()
    # disconnect from server
    db.close()

In [44]:
class Employee:
    __first_name = None
    __last_name = None
    __age = None
    __sex = False
    __onboard_date=None

    def __init__(self, first_name, last_name,age,sex):
        self.__first_name = first_name
        self.__last_name = last_name
        self.__age = age
        self.__sex = sex
        
    def get_first_name(self):
        return self.__first_name
        
    def get_last_name(self):
        return self.__last_name
    
    def set_age(self, age):
        self.__age = age

    def get_age(self):
        return self.__age

    def get_sex(self):
        return self.__sex
    
    def set_onboard_date(self, onboard_date):
        self.__onboard_date = onboard_date

    def get_onboard_date(self):
        return self.__onboard_date
    

In [45]:
import names
import random
import datetime
# employee = Employee(names.get_first_name(), names.get_last_name(),random.randint(16,35),"M")
# employee.set_onboard_date(datetime.datetime.now())
# create_employee(employee)
# for e in read_employee_list():
#     print(e)

We want to define a test job
* We want to do an intergration test

As a admin, we want to create a user in the system successfully
Steps:
* Creates a user in the system
* Read from the database 
* Validate the name matches we just set
* Report auto generated in html format


In [63]:
def create_test_record(test_case_name, status,log,employee_id, created_at):
    db = pymysql.connect(domain, user, password, db_name)
    table_name="report"

    cursor = db.cursor()

    # Prepare SQL query to INSERT a record into the database.
    #INSERT INTO `automation`.`report` (`test_case_name`, `status`, `employee_id`) VALUES ('create employee', 'success', '1');

    sql = "INSERT INTO %s (`test_case_name`,`status`,`log`,`employee_id`, `created_at`) VALUES ('%s', '%s', '%s','%d', '%s')" \
          % (table_name, test_case_name, status,log,employee_id, created_at)
    # try:
    # Execute the SQL command
    cursor.execute(sql)

    cursor.close()
    db.commit()
    # disconnect from server
    db.close()
def read_tests_list():
    db = pymysql.connect(domain, user, password, db_name)
    # prepare a cursor object using cursor() method
    cursor = db.cursor()
    # Prepare SQL query to INSERT a record into the database.
    sql = "SELECT * FROM automation.report"

    # Execute the SQL command
    cursor.execute(sql) 
    # disconnect from server
    db.close()
    # Fetch all the rows in a list of lists.
    return cursor.fetchall()

def record_to_html_table(data, delimiter=","):
    header = '''
    <!DOCTYPE html>
        <html>
        <head>
        <style type="text/css">
        .tg  {border-collapse:collapse;border-spacing:0;border-color:#9ABAD9;}
        .tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#9ABAD9;color:#444;background-color:#EBF5FF;}
        .tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#9ABAD9;color:#fff;background-color:#409cff;}
        .tg .tg-phtq{background-color:#D2E4FC;border-color:inherit;text-align:left;vertical-align:top}
        .tg .tg-jdbj{background-color:#34ff34;border-color:inherit;text-align:right;vertical-align:top}
        .tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
        .tg .tg-ydyv{background-color:#D2E4FC;border-color:inherit;text-align:right;vertical-align:top}
        .tg .tg-dw5h{background-color:#fe0000;border-color:inherit;text-align:right;vertical-align:top}
        .tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
        .tg .tg-dvpl{border-color:inherit;text-align:right;vertical-align:top}
        </style>
        </head>
        <body>
    '''
    table = header + '<table class="tg">'
    table += ''.join(['<th class="tg-c3ow" colspan="6">Test Report</th>'])
    table+='''  <tr>
    <td class="tg-hmp3">Test Case ID</td>
    <td class="tg-hmp3">Test Case Name</td>
    <td class="tg-hmp3">Status</td>
    <td class="tg-hmp3">Log</td>
    <td class="tg-hmp3">Tester</td>
    <td class="tg-hmp3">Test Complete Time</td>
  </tr>'''
    # Converting csv to html row by row
    for row in data:
        table += '<tr>'
        for cell in row:
            if cell=='Pass':
                table += '<td class="tg-jdbj">' + str(cell) + '</td>\n'
            elif cell=='Failed':
                table += '<td class="tg-dw5h">' + str(cell) + '</td>\n'
            else:
                table += '<td class="tg-dvpl">' + str(cell) + '</td>\n'
    table += '<tr></table>'
    footer = '''
        </body>
        </html>
            '''
    return table + footer

In [64]:
import unittest
import HtmlTestRunner
class TestEmployee(unittest.TestCase):
    def test_create_user(self):
        employee = Employee(names.get_first_name(), names.get_last_name(),random.randint(16,35),"M")
        employee.set_onboard_date(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        create_employee(employee)
        print(employee.get_first_name())
        for e in read_employee_list():
            if e[5]!=None :
                if e[5].strftime("%Y-%m-%d %H:%M:%S")==employee.get_onboard_date():
                    if employee.get_first_name()!= e[1]+'a':
                        create_test_record('test_create_user','Failed',\
                                           employee.get_first_name()+' not match '+e[1],1,datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    else:
                        create_test_record('test_create_user','Pass','',1,datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
        # Converting to html
    html_table = record_to_html_table(read_tests_list())
    text_file = open('report-' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ".html", "w")
    text_file.write(html_table)
    text_file.close()

John


.
----------------------------------------------------------------------
Ran 1 test in 0.438s

OK
